In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from datetime import datetime
from ipynb.fs.full.financial_graphs import price_hist as price_graph
from ipynb.fs.full.financial_graphs import normalized_graph as n_graph
from ipynb.fs.full.financial_graphs import eff_frontier as eff
from ipynb.fs.full.generate_portfolios import generatePortfolios as generate

In [ ]:
# Set start and end date for the financial data
today = datetime.today().strftime('%Y-%m-%d')
startDate = '2017-1-1'

In [ ]:
# Select tickers
equities = ['AMZN','LULU']
equityPrices = pd.DataFrame()

for e in equities:
    equityPrices[e] = wb.DataReader(e,data_source='yahoo',start=startDate,end=today)['Adj Close']
    
# Call graph functions
price_graph(equityPrices), n_graph(equityPrices)

In [ ]:
# Calculate returns (log returns) and volatility/risk (standard deviation) for each equity. Set risk-free rate
rf = 0.01
dailyReturns = np.log(1 + equityPrices.pct_change())
excessReturns = dailyReturns.mean()*252 - rf
standardDeviation = dailyReturns.std() * np.sqrt(252)


# Build covariance matrix to show relationships between equities.  Left to right diagonal is equity's variance.
covMatrix = dailyReturns.cov() * 252
numEquities = len(equities)

# Call generatePortfolios
returns,risk,weights = generate(excessReturns, standardDeviation,covMatrix,equities)

In [ ]:
# Populate dataframe with return and risk for each portfolio along with its respective weights for each equity.
portfolios = pd.DataFrame({**{'Risk_Premium':returns,'Standard_Deviation':risk},**weights})
portfolios.eval('Sharpe_Ratio=Risk_Premium/Standard_Deviation',inplace = True)
firstColumn = portfolios.pop('Sharpe_Ratio')
portfolios.insert(0,'Sharpe_Ratio',firstColumn)

In [ ]:
# Sort portfolios by Sharpe_Ratio descending. Puts the optimal portfolio of the set at the top of the data frame
portfolios.sort_values(by='Sharpe_Ratio',ascending=False)

In [ ]:
# Assign index of optimal portfolio to variable to plot on a graph.
optimalPortfolio = int(portfolios.idxmax('index')['Sharpe_Ratio'])

# Call function to graph efficient frontier
eff(portfolios,optimalPortfolio,risk,returns)